**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "silah" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpeg")
print(images_list)

['data/obj/a90.jpg', 'data/obj/a24.jpg', 'data/obj/b66.jpg', 'data/obj/32.jpg', 'data/obj/b53.jpg', 'data/obj/a100.jpg', 'data/obj/a70.jpg', 'data/obj/b16.jpg', 'data/obj/25.jpg', 'data/obj/a10.jpg', 'data/obj/a4.jpg', 'data/obj/a69.jpg', 'data/obj/a27.jpg', 'data/obj/a73.jpg', 'data/obj/a1.jpg', 'data/obj/b8.jpg', 'data/obj/a29.jpg', 'data/obj/a49.jpg', 'data/obj/b7.jpg', 'data/obj/a67.jpg', 'data/obj/b22.jpg', 'data/obj/a37.jpg', 'data/obj/a71.jpg', 'data/obj/a97.jpg', 'data/obj/a91.jpg', 'data/obj/3.jpg', 'data/obj/a55.jpg', 'data/obj/b61.jpg', 'data/obj/a31.jpg', 'data/obj/b68.jpg', 'data/obj/22.jpg', 'data/obj/a47.jpg', 'data/obj/a78.jpg', 'data/obj/a86.jpg', 'data/obj/b44.jpg', 'data/obj/a54.jpg', 'data/obj/b19.jpg', 'data/obj/a98.jpg', 'data/obj/a59.jpg', 'data/obj/7.jpg', 'data/obj/b37.jpg', 'data/obj/b10.jpg', 'data/obj/b58.jpg', 'data/obj/14.jpg', 'data/obj/b35.jpg', 'data/obj/b20.jpg', 'data/obj/b82.jpg', 'data/obj/30.jpg', 'data/obj/a94.jpg', 'data/obj/b40.jpg', 'data/obj/b

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show